# Assignment 4 

In [1]:
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt
import animatplot as amp
from matplotlib.animation import FuncAnimation

from PIL import Image
from IPython.display import Math, display

%config InlineBackend.figure_format = 'pdf'

## Finite Difference Time Domain method